In [8]:
import numpy as np

def func(x, s):
    return x**3 * np.exp( - 0.5 * x**2 / s**2 ) 

I = 10000.0
N = 100000000
s = 10

# x = np.arctan( np.random.uniform(low=0.0, high=1.0, size=N) )

x = np.random.uniform(low=0.0, high=I, size=N)

y = func( x , s )

print(y.size)

integral = I * np.sum(y) / N

print("Monte Carlo integral = {:.3f}".format(integral))

print("Theoretical integral = {:.3f}".format(2 * s**4))

100000000
Monte Carlo integral = 19899.958
Theoretical integral = 20000.000


In [15]:
import matplotlib.pyplot as plt

d = [0, 1, 2, 3, 4]
c = [109, 65, 22, 3, 1]

